# Police Investigation Simulation Class

This is base code that we used to simulate our police investigations through the networks. The Investigation class contains all the methods to search through the network which allows the user to set a model of probabilities and then implement a strategy of their choosing. This file is an example of what we did. The plot at the end of this file is a nice visualization of the process. We took this and put it through many simulations to gather our results. The models and strategies folder contains the other strategies we employed. 

In [21]:
import matplotlib
import networkx as nx
import numpy as np
import pickle
import os
from functools import partial
import matplotlib.pyplot as plt
from time import sleep
from typing import Callable, DefaultDict, Sequence
from random import choice

class Investigation():
    def __init__(self, crime_network:nx.Graph, random_catch=0.05, first_criminal:int = None, compute_eigen = True, title:str = "", caught_color:str="black",
        suspect_color:str="red", criminal_color:str="blue", informed_color:str="orange", non_gc = False):
        '''
        Class to handle investigations simulations of criminal networks. Main method is to either call investigate or simulation to run investigate in a loop.
        the crime network will be initiated with node attributes: "suspected" and "caught, and edge attribute: "informed". 

        Args:
            crime_network: Undirected, weighted graph. Optional graph property "name" will be used for plotting.
            random_catch: Base probability of catching a criminal. Either a float (uniform for all criminals) or a 
                dictionary of probabilties {node: probability}
            first_criminal: Optionally initialize the first criminal otherwise a random criminal will be used
            compute_eigen: Computes and stores the eigenvector centrality in handle "eigen" upon init. 
            title: Title used for plotting of graph.
            informed_color, caught_color, suspect_color, criminal_color: colors for plotting nodes and edges.
            non_gc: Default is False. If True, will not convert graph to giant component.

        Methods:
            set_model: Set the underlying model.
            set_strategy: Set the underlying strategy.
            investigate: Attempt to catch a criminal using the set strategy for the current investigation. 
            simulate: Run multiple investigations to a stopping criterion.
            reset: Resets the network but keeps the set model and strategy.
            plot: Plot the network.
            set_layout: Set plotting layout.
        
        Attributes:
            investigations, caught, suspects, log, fig, ax, title. 
        '''
        #Network intiializationa and attributes
        components = list(nx.connected_components(crime_network))
        if len(components) != 1:
            crime_network = crime_network.subgraph(max(components))
            print("Graph had multiple components, keeping only giant component")
        del components
        self.crime_network = crime_network
        nx.set_node_attributes(self.crime_network, False, "suspected")
        nx.set_node_attributes(self.crime_network, False, "caught")
        nx.set_edge_attributes(self.crime_network, False, "informed")
        nx.set_node_attributes(self.crime_network, criminal_color, "color")
        nx.set_edge_attributes(self.crime_network, informed_color, "color")
        nx.set_node_attributes(self.crime_network, 0, "investigations")
        self.eigen = False
        self.total_eigen = None
        if compute_eigen:
            self._compute_eigen_centrality()
        
        if isinstance(random_catch, float):
            assert 0<= random_catch <= 1, f"Random Catch needs to be a valid probability but got {random_catch}"
            self.random_catch = random_catch
        elif isinstance(random_catch, dict):
            assert set(random_catch.keys()) == set(self.crime_network.nodes), f"Random Catch dictionary does not include all nodes."
            assert all([isinstance(x, (float, int)) for x in random_catch.values()]), f"Random Catch dictionary can only be int or float"
            assert all(np.array(list(random_catch.values())) <= 1) and all(np.array(list(random_catch.values())) >= 0), "Not all probabilities are valid probabilities"
            self.random_catch = random_catch
        else:
            raise Exception(f"Need to pass either a float or dict for random_catch probabilities, instead got {random_catch}")

        self.investigations = 1
        self.caught = []
        self.suspects = []
        self.log = DefaultDict(list)

        self.current_investigation = None
        if first_criminal == None:
            first_criminal = choice(list(self.crime_network.nodes))

        self.model_proba = None
        self.strategy = None
        
        #Plotting attributes
        if title == "":
            try:
                self.title = self.crime_network.graph["name"]
            except KeyError:
                print("Graph had no name property for titling plots")
        else:
            self.title = title
        self.criminal_color = criminal_color
        self.suspect_color = suspect_color
        self.caught_color = caught_color
        self.informed_color = informed_color

        self.fig = None
        self.ax = None
        self.layout = nx.layout.spring_layout(self.crime_network, k = 0.5 / np.sqrt(len(self.crime_network.nodes)))

        #Initialize
        self._caught_suspect(first_criminal)
        self._log_stats()

    def _compute_eigen_centrality(self, handle = "eigen"):
        ec = nx.eigenvector_centrality_numpy(self.crime_network)
        nx.set_node_attributes(self.crime_network, ec, name=handle)
        self.eigen = True
        self.total_eigen = np.sum(list(ec.values()))

    def _model_check(self):
        if self.strategy is None:
            print("No strategy set. Please set a strategy using set_strategy method.")
            return False
        if self.model_proba is None: 
            print("No underlying model is defined. Please define model using set_model method.")
            return False

    def _set_probas(self, suspect_probas:dict={}):
        '''Set new capture probabilities'''
        nx.set_node_attributes(self.crime_network, self.random_catch, name="random_catch")
        caught_probas = {node : 0 for node, attr in self.crime_network.nodes.data() if attr["caught"] == True}
        nx.set_node_attributes(self.crime_network, suspect_probas, name="catch_proba")
        nx.set_node_attributes(self.crime_network, caught_probas, name="catch_proba")

    def _catch_random(self):
        '''Catch a random unsuspected criminal'''
        unsuspected = [node for node, suspected in list(self.crime_network.nodes(data="suspected")) if not suspected]
        candidate = choice(unsuspected)
        if np.random.uniform < self.crime_network.nodes[candidate]["random_catch"]:
            self._caught_suspect(candidate, random = True)

    def _caught_suspect(self, suspect:int, random = False):
        '''Update graph properties when suspect is caught'''
        #Update the caught suspect's attributes
        self.crime_network.nodes[suspect]["caught"] = True
        self.caught.append(suspect)
        self.crime_network.nodes[suspect]["color"] = self.caught_color
        self.crime_network.nodes[suspect]["suspected"] = False
        if self.investigations != 1 or random:
            self.suspects.remove(suspect)
        for i, j in list(self.crime_network.edges(suspect)):    
            #Use provided order to choose source-target
            if j not in self.caught and j not in self.suspects: #Could I just check if not an informed edge????
                self.crime_network.nodes[j]["suspected"] = True
                self.suspects.append(j)
                self.crime_network.nodes[j]["color"] = self.suspect_color
            #Reorder edges to index edges
            i, j = min(i, j), max(i, j) #Unecessary line?
            self.crime_network[i][j]["informed"] = True
            self.crime_network[i][j]["color"] = self.informed_color
        self._update_investigation()

    def _update_investigation(self):
        '''Update current investigation graphview with new suspected and informed'''
        def filter_node(x):
            return x in self.suspects or x in self.caught
        def filter_edge(i, j):
            return self.crime_network[i][j].get("informed", False) or self.crime_network[j][i].get("informed", False)
        self.current_investigation = nx.subgraph_view(self.crime_network, filter_node=filter_node, filter_edge=filter_edge)
    
    def _log_stats(self):
        self.log["caught"].append(len(self.caught))
        self.log["suspects"].append(len(self.suspects))
        self.log["informed"].append(len(self.current_investigation.edges))
        if self.eigen:
            self.log["captured_eigen"].append(np.sum([self.crime_network.nodes.data("eigen")[i] for i in self.caught]))
            self.log["eigen_proportion"].append(np.sum([self.crime_network.nodes.data("eigen")[i] for i in self.caught])/self.total_eigen)
        self.log["investigation"].append(self.investigations)

    def set_model(self, model:Callable, **kwargs):
        '''Set underlying probability model for capture of suspects. Function should take a graph of suspects and known criminals as an argument
            and return a dictionary of probabilities corresponding to {suspect: probability}'''
        self.model_proba = partial(model, self.current_investigation, **kwargs)

    def set_strategy(self, strategy:Callable, **kwargs):
        '''Sets investigation strategy. Function should take at least a graph of suspects and known criminals as an argument and return a 
            suspect (int) or string "random" in place of the suspect to catch a random unknown criminal instead.'''
        self.strategy =  partial(strategy, **kwargs)
    
    def set_layout(self, pos):
        '''Set network x plotting layout.'''
        self.layout = pos

    def investigate(self, plot:bool = False, update_plot:bool = False, **plot_kwargs):
        '''
        Makes an attempt to catch a criminal according to strategy.

        Args:
            plot: If true calls plot method. 
            update_plot: If true attempts to update stored plot interactively. If no plot has been made, will call plot method normally.
        '''
        if self._model_check() == False:
            return
        #Set probabilities of the model
        suspect_probas = self.model_proba()
        self._set_probas(suspect_probas)
        suspect = self.strategy(self.current_investigation)
        
        if suspect == "random":
            self._catch_random()
        elif np.random.uniform() < self.crime_network.nodes[suspect]["catch_proba"] \
            + self.crime_network.nodes[suspect]["random_catch"]:
            self._caught_suspect(suspect)
            self.crime_network.nodes[suspect]["investigations"] += 1
        elif np.random.uniform() >= self.crime_network.nodes[suspect]["catch_proba"] \
            + self.crime_network.nodes[suspect]["random_catch"]:
            self.crime_network.nodes[suspect]["investigations"] += 1
        self.investigations += 1
        self._log_stats()

        if update_plot:
            if self.ax and self.fig:
                if not matplotlib.is_interactive():
                    plt.ion()
                self.fig.canvas.flush_events()
                self.ax.clear()
                self.plot(**plot_kwargs)
                self.fig.canvas.draw()
            else:
                self.plot(**plot_kwargs)
                self.fig.show()
        elif plot:
            self.plot(**plot_kwargs)

    def simulate(self, max_criminals:int = 0, max_investigations:int = 0, update_plot:bool = False, sleep_time:float = 0.5, **kwargs):
        '''
        Investigates until either stopping criterion is met or entire network caught.

            Args:
                max_criminals: Number of criminals to catch before stopping investigation.
                max_investigations: Number of investigations to make before stopping investigation.
                condition: "and" or "or". How stopping criterion considers combines max_criminals and max_investigations conditions. 
                update_plot: Plots and updates as simulation runs. See investigate method.
                sleep_time: Sleep time between plot updates. Default 0.5.
                **kwargs: Arguments such as plot and update_plot to be passed to investigate. 
        '''
        if self._model_check == False:
            return
        while len(self.caught) < max_criminals and self.investigations < max_investigations:
            if len(self.caught) == len(self.crime_network.nodes):
                break
            self.investigate(update_plot=update_plot, **kwargs)
            if update_plot:
                plt.pause(sleep_time)

    def plot(self, investigation_only = False, weighted:bool = True, weight_multiplier:float = 3, showfig:bool = True, label = "", **kwargs):
        '''
        Plots the network and saves to self.fig and self.ax. 

        Args:
            weighted: If true, plots weighted edge widths
            weight_multiplier: Adjusts the scale of weighted edge widths
            showfig: If true, calls fig.show(). 
            label: Text label under title.
            **kwargs: Extra arguments passed to nx.draw()
        '''
        g = self.current_investigation if investigation_only else self.crime_network

        if weighted:
            weights = np.array(list(nx.get_edge_attributes(self.crime_network, "weight").values()))
            weights = (weights - weights.min()+1)/(weights.max()+1) * weight_multiplier
            kwargs.update({"width":weights})
        
        if not self.ax:
            self.fig, self.ax = plt.subplots(figsize=(20, 20))

        statistics = f"Investigations: {self.investigations}\nCaught Criminals: {len(self.caught)}\nSuspects: {len(self.suspects)}"
        
        if self.eigen:
            captured_eigen = np.sum([g.nodes.data("eigen")[i] for i in self.caught])
            statistics = statistics + f"\nCaptured EC Proportion:{round(captured_eigen/self.total_eigen, 2)}"

        nx.draw(g, pos=self.layout, 
            ax=self.ax, node_color = dict(g.nodes.data("color")).values(),
            edge_color = [color for _, _, color in g.edges.data("color")],
            **kwargs)
        self.ax.set_axis_off()
        self.ax.set_title(self.title, fontsize=30)
        self.ax.text(x=0, y = 0, s=label, fontsize=15, transform=self.ax.transAxes)
        self.ax.text(x = 0.8, y = 0, 
            s = statistics,
            transform=self.ax.transAxes, fontsize=15)
        
        if showfig:
            self.fig.show()
    
    def reset(self, first_criminal:int = None, keep_fig:bool = False, verbose=False):
        '''
        Resets the network, and reinitializes. Does not reset the model and strategy.
        
        Args:
            first_criminal: Initialize with an optional first criminal index. Otherwise random.
            keep_fig: If true, does not reset the current figure and ax. Set to true and run subsequent simulations using update_plot
                to keep refreshing to a new figure
        '''
        nx.set_node_attributes(self.crime_network, False, "suspected")
        nx.set_node_attributes(self.crime_network, False, "caught")
        nx.set_edge_attributes(self.crime_network, False, "informed")
        nx.set_edge_attributes(self.crime_network, 0, "investigations")
        nx.set_node_attributes(self.crime_network, self.criminal_color, "color")
        nx.set_edge_attributes(self.crime_network, self.informed_color, "color")
        self.investigations = 1
        self.caught = []
        self.suspects = []
        self.log = DefaultDict(list)        

        self.current_investigation = None
        if first_criminal == None:
            first_criminal = choice(list(self.crime_network.nodes))

        if not keep_fig:
            self.fig = None
            
        self._caught_suspect(first_criminal)
        self._log_stats()
        if verbose:
            print("Crime network reset.")
    
    def refresh_fig(self):
        '''Refreshes the figure.'''
        if self.ax and self.fig:
            if not matplotlib.is_interactive():
                plt.ion()
            self.fig.canvas.flush_events()
            self.ax.clear()
            self.plot(showfig=False)
            self.fig.canvas.draw()

In [22]:
#HELPER FUNCTIONS

def get_suspects(graph:nx.Graph) -> list:
    '''Returns a list of suspects in the graph'''
    return [suspect for suspect in graph.nodes if graph.nodes[suspect].get("suspected")]

def get_caught_criminals(graph:nx.Graph) -> list:
    '''Returns a list of caught criminals in the graph'''
    return [caught for caught in graph.nodes if graph.nodes[caught].get("caught")]

def get_information(graph:nx.Graph, suspects:list, weighted:bool = True) -> dict:
    '''Returns a list of degree for each suspect. If weighted, it will sum the edge weights, otherwise take the number
    of edges activated incident to each suspect.'''
    return [degree for _, degree in graph.degree(suspects, weight=lambda _: "weight" if weighted else None)]

def get_investigations(graph:nx.Graph, suspects:list) -> dict:
    '''Returns a list with number of investigations for each suspect.'''
    investigations = nx.get_node_attributes(graph, "investigations")
    investigations_suspect = {suspect: investigations[suspect] for suspect in suspects}
    return list(investigations_suspect.values())

def get_nearby_suspects(graph:nx.Graph, centroids: list):
    '''Gets all neighboring suspects of centroids'''
    candidates = []
    for center in centroids:
        candidate_suspects = [suspect for suspect in graph.neighbors(center) \
            if graph.nodes[suspect].get("suspected") and graph[center][suspect]["informed"]]
        candidates.extend(candidate_suspects)
    return candidates

def get_connected_centrality(graph, suspect, weighted, mode="eigen"):
    '''For the suspect return the sum of all connected criminals' eigenvector centralities'''
    if mode == "eigen":
        centrality_dict = nx.eigenvector_centrality_numpy(graph, weight=lambda _: "weight" if weighted else None)
    elif mode =="degree":
        centrality_dict = dict(graph.degree)
    connected_criminals = [linked for linked in graph.neighbors(suspect) \
        if graph.nodes[linked].get("caught") and graph[suspect][linked].get("informed")]
    centrality = [centrality_dict[connected] for connected in connected_criminals]
    return np.sum(centrality)

def get_potential_diam(graph, suspects, caught):
    suspected_diameter = {}
    for suspect in suspects:
        potential_graph = nx.subgraph_view(graph, filter_node = lambda x: True if x in caught or x== suspect else False)
        suspected_diameter[suspect] = nx.diameter(potential_graph)
    return suspected_diameter


In [23]:


def constant_model(graph:nx.Graph, c:float, weighted:bool=True):
    '''Each informative link adds a constant amount of information'''

    assert 0 <= c <= 1, f"c needs to be a valid probability, instead got {c}"

    suspects = get_suspects(graph)
    information = get_information(graph, suspects, weighted)
    suspect_proba = {suspect : min(1, c * info) for suspect, info in zip(suspects, information)}
    return suspect_proba


def greedy(graph:nx.Graph, tiebreaker = "random", weighted:bool = True):
    '''Greedy search and break ties by maximum diameter of caught criminals
    Args:
        graph: Graph of the current investigation.
        tiebreaker: Accepts "random", "eigenvector", "diameter", "triangles". 
            random: Breaks ties at random.
            eigenvector: Breaks tie wiht highest perceived eigenvector centrality.
            diameter: Breaks tie by the maximum diameter of the graph of criminals if the considered suspect is added.
            triangles: Breaks tie by the maximum number of triangles of the graph of criminals if the considered suspect is added.
        weighted: Consider the graph weighted or not.
    Returns:
        Suspect index.
    '''
    assert tiebreaker in ["random", "eigenvector", "diameter", "triangles"], \
        f"Invalid tiebreaker strategy. Got {tiebreaker} and expected one of: random, eigenvector, triangles"

    suspects = get_suspects(graph)
    information = get_information(graph, suspects, weighted)

    #Filter for greediest choice
    candidate_suspects = [suspect for suspect, info in zip(suspects, information) if info == max(information)]

    #Get potential diameter if the suspect is included in the caught_criminals
    if tiebreaker == "random":
        return choice(candidate_suspects)
    elif tiebreaker=="diameter":
        caught = get_caught_criminals(graph)
        tiebreak_dict = get_potential_diam(graph, candidate_suspects, caught)
    elif tiebreaker == "eigenvector":
        try:
            tiebreak_dict = nx.eigenvector_centrality_numpy(graph, weight=lambda _: "weight" if weighted else None)
        except TypeError:
            tiebreak_dict = nx.eigenvector_centrality(graph, weight=lambda _: "weight" if weighted else None)
    elif tiebreaker == "triangles":
        tiebreak_dict = nx.triangles(graph)

    tiebreak_dict = {candidate: tiebreak_dict[candidate] for candidate in candidate_suspects}
    final_candidates = [candidate for candidate, score in tiebreak_dict.items() 
        if np.abs(score - max(tiebreak_dict.values())) <= 1e-4]

    #Random selection if still multiple
    return choice(final_candidates)


In [27]:
#Set matplotlib backend for interactive plotting
matplotlib.use("TkAgg")

#Load Network
with open("giant_component_crime_networks.pkl", "rb") as infile:
    network_graphs = pickle.load(infile)

#Instantiate model and set a constant model as described in the report. Sets a greedy diameter strategy. 
#Probability of randomly catching a criminal is 0.05, probability gained from an activated edge is c = 0.05
#Network_graphs[11] corresponds to Al Qaeda, but others can be chosen. 
inv = Investigation(crime_network = network_graphs[11], random_catch = 0.05)
inv.set_model(constant_model, c = 0.05)
inv.set_strategy(greedy, tiebreaker="diameter")


In [28]:
#Investigation from the perspective of the police. Set investiagion_only=False to investigate with the whole graph shown. 

#Blue dots are unknown criminals, Red are suspects, Black are caught criminals. 

inv.simulate(
    max_criminals=100, 
    max_investigations=100, 
    sleep_time=0.2,
    plot=True, 
    update_plot=True, 
    investigation_only=True
)

plt.close()